### Hands on CNN
#### By MiMoTrix
##### MNIST with CNN

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


##### Initial Values

In [3]:
learning_rate = 0.001
batch_size = 128
epochs = 100

##### Net Parameters

In [5]:
num_inputs = 784 # 28*28 pixels images
num_classes = 10 # 0 to 9 digits

##### Input Placeholders

In [25]:
X = tf.placeholder(dtype=tf.float32, shape=(None, 784), name="Inputs")
Y = tf.placeholder(dtype=tf.float32, shape=(None, 10), name="Lables")
keep_prob = tf.placeholder(dtype=tf.float32) # DropOut ( Keep Probability )

##### 2d Convolution

In [26]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

##### MaxPool 2D

In [27]:
def maxPool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

##### Creating Model

In [31]:
def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    # Layer One
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxPool2d(conv1, k=2)
    # Layer Two
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxPool2d(conv2, k=2)
    # Fully Connected Layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    # Output
    output = tf.add(tf.matmul(fc1, weights['output']), biases['output'])
    return output

##### Storing Layers' Weights and Biases

In [32]:
weights = {
    # 5*5 conv, 1 input, 64 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 64])),
    # 5*5 conv, 64 input, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 64, 64])),
    # Fully Connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 500])),
    # 500 inputs, 10 outputs ( Class Prediction )
    'output': tf.Variable(tf.random_normal([500, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([64])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([500])),
    'output': tf.Variable(tf.random_normal([num_classes])),
}

##### Construct Model

In [33]:
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

##### Optimizer and Loss Definition

In [34]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

##### Evaluate Model

In [35]:
CorrectPrediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(CorrectPrediction, tf.float32))

##### Initial Variables

In [36]:
init = tf.global_variables_initializer()

##### Session

In [37]:
with tf.Session() as sess:
    sess.run(init)
    n_batches = int(mnist.train.num_examples / batch_size)
    for iteratorA in range(epochs):
        for iteratorB in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            sess.run(train_op, feed_dict={X: X_batch, Y: Y_batch, keep_prob: 0.8})
        loss, acc = sess.run([loss_op, accuracy], feed_dict={X: X_batch, Y: Y_batch, keep_prob: 1.0})
        print("Epoch ", str(iteratorA), ", MiniBatch Loss: ", "{:.4f}".format(loss), 
              ", Training Accuracy: ", "{:.4f}".format(acc))
    print("Optimization Finished :)")
    print("Testing Accuracy: ", sess.run(accuracy, feed_dict={X: mnist.test.images[:256], 
                                                              Y: mnist.test.lables[:256],
                                                              keep_prob: 1.0}))

Epoch  0 , MiniBatch Loss:  888.4835 , Training Accuracy:  0.9219


Epoch  1 , MiniBatch Loss:  341.5340 , Training Accuracy:  0.9375


Epoch  2 , MiniBatch Loss:  143.0342 , Training Accuracy:  0.9688


KeyboardInterrupt: 